# 🗺️ MCP Map Servers - Google Colab Version

This notebook provides an interactive interface to the MCP Map Servers project.

**Features:**
- Geocoding (address ↔ coordinates)
- Routing with directions
- POI search (cafes, restaurants, museums, etc.)
- Interactive widget interface

**Setup Steps:**
1. Run all cells in order
2. Enter your OpenAI API key when prompted
3. Use the interactive interface at the bottom

## 📦 Step 1: Install Dependencies

In [1]:
# Install required packages
!pip install -q openai requests gradio

print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


## 🔑 Step 2: Set API Key

**Important:** Add your OpenAI API key to Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add a new secret named `OPENAI_API_KEY`
3. Paste your API key as the value
4. Toggle "Notebook access" ON

In [2]:
import os
from google.colab import userdata

# Get OpenAI API key from Colab secrets
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets!")
except Exception as e:
    print("❌ Error: Could not find OPENAI_API_KEY in Colab secrets.")
    print("Please add it using the 🔑 key icon in the left sidebar.")
    raise

✅ API key loaded from Colab secrets!


## 📝 Step 3: Configuration Classes

In [3]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class CityNavigatorConfig:
    nominatim_base_url: str = "https://nominatim.openstreetmap.org"
    nominatim_user_agent: str = "mcp-map-servers-colab/0.1.0"
    osrm_base_url: str = "https://router.project-osrm.org"
    timeout: int = 30
    max_retries: int = 3

@dataclass
class PlacesExplorerConfig:
    overpass_base_url: str = "https://overpass-api.de/api"
    photon_base_url: str = "https://photon.komoot.io"
    timeout: int = 30
    max_retries: int = 3
    user_agent: str = "mcp-map-servers-colab/0.1.0"

@dataclass
class AgentConfig:
    openai_api_key: str
    model: str = "gpt-4o-mini"
    temperature: float = 0.7
    max_tokens: Optional[int] = None

print("✅ Configuration classes loaded!")

✅ Configuration classes loaded!


## 🚗 Step 4: CityNavigatorServer

In [4]:
import requests
import time
from typing import Dict, List, Tuple, Union

@dataclass
class GeocodingResult:
    latitude: float
    longitude: float
    display_name: str
    address: Dict[str, str]
    place_id: Optional[str] = None

    def to_dict(self) -> dict:
        return {
            "latitude": self.latitude,
            "longitude": self.longitude,
            "display_name": self.display_name,
            "address": self.address,
            "place_id": self.place_id,
        }

@dataclass
class RouteResult:
    distance_km: float
    duration_minutes: float
    geometry: List[Tuple[float, float]]
    steps: List[str]

    def to_dict(self) -> dict:
        return {
            "distance_km": self.distance_km,
            "duration_minutes": self.duration_minutes,
            "geometry": self.geometry,
            "steps": self.steps,
        }

class CityNavigatorServer:
    def __init__(self, config: Optional[CityNavigatorConfig] = None):
        self.config = config or CityNavigatorConfig()
        self.session = requests.Session()
        self.session.headers.update({"User-Agent": self.config.nominatim_user_agent})

    def forward_geocode(self, address: str, limit: int = 1) -> List[GeocodingResult]:
        url = f"{self.config.nominatim_base_url}/search"
        params = {"q": address, "format": "json", "addressdetails": 1, "limit": limit}

        response = self.session.get(url, params=params, timeout=self.config.timeout)
        response.raise_for_status()
        time.sleep(1)  # Nominatim rate limit

        data = response.json()
        if not data:
            raise ValueError(f"No results found for address: {address}")

        return [GeocodingResult(
            latitude=float(item["lat"]),
            longitude=float(item["lon"]),
            display_name=item.get("display_name", ""),
            address=item.get("address", {}),
            place_id=item.get("place_id"),
        ) for item in data]

    def reverse_geocode(self, latitude: float, longitude: float) -> GeocodingResult:
        url = f"{self.config.nominatim_base_url}/reverse"
        params = {"lat": latitude, "lon": longitude, "format": "json", "addressdetails": 1}

        response = self.session.get(url, params=params, timeout=self.config.timeout)
        response.raise_for_status()
        time.sleep(1)

        data = response.json()
        if "error" in data:
            raise ValueError(f"No results found for coordinates: {latitude}, {longitude}")

        return GeocodingResult(
            latitude=float(data["lat"]),
            longitude=float(data["lon"]),
            display_name=data.get("display_name", ""),
            address=data.get("address", {}),
            place_id=data.get("place_id"),
        )

    def route(self, origin: Union[str, Tuple[float, float]],
              destination: Union[str, Tuple[float, float]], mode: str = "driving") -> RouteResult:
        # Convert addresses to coordinates if needed
        if isinstance(origin, str):
            origin_coords = self.forward_geocode(origin, limit=1)[0]
            origin = (origin_coords.latitude, origin_coords.longitude)
        if isinstance(destination, str):
            dest_coords = self.forward_geocode(destination, limit=1)[0]
            destination = (dest_coords.latitude, dest_coords.longitude)

        profile = {"driving": "car", "walking": "foot", "cycling": "bike"}.get(mode.lower(), "car")
        url = f"{self.config.osrm_base_url}/route/v1/{profile}/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
        params = {"overview": "full", "steps": "true", "geometries": "geojson"}

        response = self.session.get(url, params=params, timeout=self.config.timeout)
        response.raise_for_status()
        data = response.json()

        if data.get("code") != "Ok":
            raise ValueError(f"Routing failed: {data.get('message', 'Unknown error')}")

        route = data["routes"][0]
        geometry = [(coord[1], coord[0]) for coord in route["geometry"]["coordinates"]]
        steps = [step["maneuver"].get("instruction", "")
                for leg in route["legs"] for step in leg["steps"]
                if "maneuver" in step and "instruction" in step["maneuver"]]

        return RouteResult(
            distance_km=route["distance"] / 1000,
            duration_minutes=route["duration"] / 60,
            geometry=geometry,
            steps=steps,
        )

    def get_tool_definitions(self) -> List[dict]:
        return [
            {"type": "function", "function": {"name": "forward_geocode",
             "description": "Convert address to coordinates",
             "parameters": {"type": "object", "properties": {
                 "address": {"type": "string"}}, "required": ["address"]}}},
            {"type": "function", "function": {"name": "reverse_geocode",
             "description": "Convert coordinates to address",
             "parameters": {"type": "object", "properties": {
                 "latitude": {"type": "number"}, "longitude": {"type": "number"}},
                 "required": ["latitude", "longitude"]}}},
            {"type": "function", "function": {"name": "route",
             "description": "Calculate route between two points",
             "parameters": {"type": "object", "properties": {
                 "origin": {"type": "string"}, "destination": {"type": "string"},
                 "mode": {"type": "string", "enum": ["driving", "walking", "cycling"]}},
                 "required": ["origin", "destination"]}}}
        ]

print("✅ CityNavigatorServer loaded!")

✅ CityNavigatorServer loaded!


## 📍 Step 5: PlacesExplorerServer

In [5]:
import math

@dataclass
class Place:
    name: str
    category: str
    latitude: float
    longitude: float
    address: Optional[str] = None
    distance_km: Optional[float] = None
    tags: Optional[Dict[str, str]] = None

    def to_dict(self) -> dict:
        return {
            "name": self.name, "category": self.category,
            "latitude": self.latitude, "longitude": self.longitude,
            "address": self.address, "distance_km": self.distance_km, "tags": self.tags
        }

class PlacesExplorerServer:
    def __init__(self, config: Optional[PlacesExplorerConfig] = None):
        self.config = config or PlacesExplorerConfig()
        self.session = requests.Session()
        self.session.headers.update({"User-Agent": self.config.user_agent})
        self.category_map = {
            "cafe": "amenity=cafe", "restaurant": "amenity=restaurant",
            "bar": "amenity=bar", "hotel": "tourism=hotel",
            "museum": "tourism=museum", "park": "leisure=park",
            "pharmacy": "amenity=pharmacy", "hospital": "amenity=hospital",
            "bank": "amenity=bank", "supermarket": "shop=supermarket"
        }

    def _geocode_city(self, city: str) -> Tuple[float, float]:
        url = f"{self.config.photon_base_url}/api"
        response = self.session.get(url, params={"q": city, "limit": 1}, timeout=self.config.timeout)
        response.raise_for_status()
        data = response.json()
        if not data.get("features"):
            raise ValueError(f"Could not geocode city: {city}")
        coords = data["features"][0]["geometry"]["coordinates"]
        return (coords[1], coords[0])

    def search_poi(self, category: str, city: str, limit: int = 10) -> List[Place]:
        lat, lon = self._geocode_city(city)
        overpass_tag = self.category_map.get(category.lower(), f"amenity={category.lower()}")

        query = f"""[out:json][timeout:25];
        (node[{overpass_tag}](around:5000,{lat},{lon});
         way[{overpass_tag}](around:5000,{lat},{lon}););
        out center {limit};"""

        response = self.session.post(f"{self.config.overpass_base_url}/interpreter",
                                    data={"data": query}, timeout=self.config.timeout)
        response.raise_for_status()
        elements = response.json().get("elements", [])

        places = []
        for elem in elements[:limit]:
            if elem["type"] == "node":
                elem_lat, elem_lon = elem["lat"], elem["lon"]
            else:
                center = elem.get("center", {})
                elem_lat, elem_lon = center.get("lat"), center.get("lon")

            if not elem_lat or not elem_lon:
                continue

            tags = elem.get("tags", {})
            places.append(Place(
                name=tags.get("name", f"Unnamed {category}"),
                category=category, latitude=elem_lat, longitude=elem_lon,
                address=tags.get("addr:full", ""),
                distance_km=self._calculate_distance(lat, lon, elem_lat, elem_lon),
                tags=tags
            ))
        return places

    def search_nearby(self, latitude: float, longitude: float,
                     category: Optional[str] = None, radius_km: float = 1.0, limit: int = 10) -> List[Place]:
        radius_m = int(radius_km * 1000)

        if category:
            tag = self.category_map.get(category.lower(), f"amenity={category.lower()}")
            query = f"""[out:json][timeout:25];
            (node[{tag}](around:{radius_m},{latitude},{longitude}););
            out center {limit};"""
        else:
            query = f"""[out:json][timeout:25];
            (node["amenity"](around:{radius_m},{latitude},{longitude}););
            out center {limit};"""

        response = self.session.post(f"{self.config.overpass_base_url}/interpreter",
                                    data={"data": query}, timeout=self.config.timeout)
        response.raise_for_status()
        elements = response.json().get("elements", [])

        places = []
        for elem in elements[:limit]:
            elem_lat = elem.get("lat") or elem.get("center", {}).get("lat")
            elem_lon = elem.get("lon") or elem.get("center", {}).get("lon")
            if not elem_lat or not elem_lon:
                continue

            tags = elem.get("tags", {})
            places.append(Place(
                name=tags.get("name", "Unnamed location"),
                category=tags.get("amenity", category or "unknown"),
                latitude=elem_lat, longitude=elem_lon,
                distance_km=self._calculate_distance(latitude, longitude, elem_lat, elem_lon),
                tags=tags
            ))

        places.sort(key=lambda p: p.distance_km or float('inf'))
        return places

    def suggest_places(self, query: str, limit: int = 5) -> List[Place]:
        response = self.session.get(f"{self.config.photon_base_url}/api",
                                   params={"q": query, "limit": limit}, timeout=self.config.timeout)
        response.raise_for_status()
        features = response.json().get("features", [])

        return [Place(
            name=f.get("properties", {}).get("name", "Unnamed location"),
            category=f.get("properties", {}).get("type", "location"),
            latitude=f["geometry"]["coordinates"][1],
            longitude=f["geometry"]["coordinates"][0],
            tags=f.get("properties", {})
        ) for f in features if len(f.get("geometry", {}).get("coordinates", [])) >= 2]

    def _calculate_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        R = 6371
        lat1_rad, lat2_rad = math.radians(lat1), math.radians(lat2)
        delta_lat, delta_lon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
        a = math.sin(delta_lat/2)**2 + math.cos(lat1_rad)*math.cos(lat2_rad)*math.sin(delta_lon/2)**2
        return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    def get_tool_definitions(self) -> List[dict]:
        return [
            {"type": "function", "function": {"name": "search_poi",
             "description": "Search POIs by category in a city",
             "parameters": {"type": "object", "properties": {
                 "category": {"type": "string"}, "city": {"type": "string"}},
                 "required": ["category", "city"]}}},
            {"type": "function", "function": {"name": "search_nearby",
             "description": "Find places near coordinates",
             "parameters": {"type": "object", "properties": {
                 "latitude": {"type": "number"}, "longitude": {"type": "number"},
                 "category": {"type": "string"}, "radius_km": {"type": "number"}},
                 "required": ["latitude", "longitude"]}}},
            {"type": "function", "function": {"name": "suggest_places",
             "description": "Get place suggestions from text query",
             "parameters": {"type": "object", "properties": {
                 "query": {"type": "string"}}, "required": ["query"]}}}
        ]

print("✅ PlacesExplorerServer loaded!")

✅ PlacesExplorerServer loaded!


## 🤖 Step 6: MapAssistant (OpenAI Agent)

In [6]:
import json
from openai import OpenAI
from typing import Any

class MapAssistant:
    def __init__(self):
        self.client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.nav_server = CityNavigatorServer()
        self.places_server = PlacesExplorerServer()
        self.tools = self.nav_server.get_tool_definitions() + self.places_server.get_tool_definitions()

        self.system_prompt = """You are a helpful map and navigation assistant.
You have access to geocoding, routing, and POI search capabilities.
Always provide clear, helpful responses with specific information from the tools."""

    def _execute_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        try:
            if tool_name == "forward_geocode":
                results = self.nav_server.forward_geocode(**arguments)
                return {"success": True, "results": [r.to_dict() for r in results]}
            elif tool_name == "reverse_geocode":
                result = self.nav_server.reverse_geocode(**arguments)
                return {"success": True, "result": result.to_dict()}
            elif tool_name == "route":
                result = self.nav_server.route(**arguments)
                return {"success": True, "result": result.to_dict()}
            elif tool_name == "search_poi":
                results = self.places_server.search_poi(**arguments)
                return {"success": True, "count": len(results), "results": [p.to_dict() for p in results]}
            elif tool_name == "search_nearby":
                results = self.places_server.search_nearby(**arguments)
                return {"success": True, "count": len(results), "results": [p.to_dict() for p in results]}
            elif tool_name == "suggest_places":
                results = self.places_server.suggest_places(**arguments)
                return {"success": True, "count": len(results), "results": [p.to_dict() for p in results]}
            else:
                raise ValueError(f"Unknown tool: {tool_name}")
        except Exception as e:
            return {"success": False, "error": str(e), "error_type": type(e).__name__}

    def chat(self, user_message: str, verbose: bool = False) -> str:
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_message}
        ]

        for iteration in range(5):
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )

            message = response.choices[0].message

            if not message.tool_calls:
                return message.content

            if verbose:
                print(f"🔧 Calling {len(message.tool_calls)} tool(s)...")

            messages.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": [{"id": tc.id, "type": tc.type,
                               "function": {"name": tc.function.name, "arguments": tc.function.arguments}}
                              for tc in message.tool_calls]
            })

            for tool_call in message.tool_calls:
                tool_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)

                if verbose:
                    print(f"   → {tool_name}({list(arguments.keys())})")

                result = self._execute_tool(tool_name, arguments)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })

        return "Maximum tool calls reached. Please try again."

print("✅ MapAssistant loaded!")

✅ MapAssistant loaded!


## 🧪 Testing Individual Tools

Let's test each tool individually to see how they work, then combine them in complex queries.

### 📍 Test 1: forward_geocode - Convert Address to Coordinates

In [7]:
# Test forward_geocode tool
print("🔍 Testing: forward_geocode")
print("=" * 60)

assistant = MapAssistant()
query = "What are the coordinates of the Eiffel Tower?"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: forward_geocode
📝 Query: What are the coordinates of the Eiffel Tower?

🔧 Calling 1 tool(s)...
   → forward_geocode(['address'])

✅ Response:
The coordinates of the Eiffel Tower are approximately:

- **Latitude:** 48.8582599
- **Longitude:** 2.2945006

It is located at 5 Avenue Anatole France, in the 7th arrondissement of Paris, France.


### 🔄 Test 2: reverse_geocode - Convert Coordinates to Address

In [8]:
# Test reverse_geocode tool
print("🔍 Testing: reverse_geocode")
print("=" * 60)

query = "What's the address of coordinates 48.8584, 2.2945?"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: reverse_geocode
📝 Query: What's the address of coordinates 48.8584, 2.2945?

🔧 Calling 1 tool(s)...
   → reverse_geocode(['latitude', 'longitude'])

✅ Response:
The address for the coordinates 48.8584, 2.2945 is:

**Avenue Gustave Eiffel, Quartier du Gros-Caillou,  
Paris 7e Arrondissement, Paris, France**  
**Postal Code:** 75007  
**Country:** France


### 🚗 Test 3: route - Get Directions Between Two Points

In [9]:
# Test route tool
print("🔍 Testing: route")
print("=" * 60)

query = "Get walking directions from the Louvre to Notre-Dame Cathedral in Paris"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: route
📝 Query: Get walking directions from the Louvre to Notre-Dame Cathedral in Paris

🔧 Calling 1 tool(s)...
   → route(['origin', 'destination', 'mode'])

✅ Response:
Walking from the Louvre to Notre-Dame Cathedral in Paris is straightforward and will take approximately 6 minutes over a distance of about 2.2 kilometers. Here’s a summary of your walking route:

1. **Start at the Louvre**: Exit the museum and head towards the Cour Napoléon.
2. **Head southeast**: Walk towards Rue de Rivoli.
3. **Stay on Rue de Rivoli**: Continue to follow Rue de Rivoli for a short distance.
4. **Turn onto Avenue de l'Opéra**: Follow Avenue de l'Opéra as it curves slightly.
5. **Cross the Seine**: Continue towards the Seine River and cross at one of the pedestrian bridges to get to Île de la Cité.
6. **Arrive at Notre-Dame Cathedral**: Follow the signs leading to Notre-Dame Cathedral.

Enjoy your visit! Let me know if you need any more assistance or details.


### 🏛️ Test 4: search_poi - Find Places by Category in a City

In [10]:
# Test search_poi tool
print("🔍 Testing: search_poi")
print("=" * 60)

query = "Search for museums in Berlin"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: search_poi
📝 Query: Search for museums in Berlin

🔧 Calling 1 tool(s)...
   → search_poi(['category', 'city'])

✅ Response:
It seems that there are currently no museums listed in Berlin. If you have any specific museums in mind or need assistance with anything else, feel free to ask!


### 📍 Test 5: search_nearby - Find Places Near Coordinates

In [11]:
# Test search_nearby tool
print("🔍 Testing: search_nearby")
print("=" * 60)

query = "Find restaurants within 2km of coordinates 52.5200, 13.4050"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: search_nearby
📝 Query: Find restaurants within 2km of coordinates 52.5200, 13.4050

🔧 Calling 1 tool(s)...
   → search_nearby(['latitude', 'longitude', 'category', 'radius_km'])
🔧 Calling 1 tool(s)...
   → search_nearby(['latitude', 'longitude', 'category', 'radius_km'])

✅ Response:
Here are some restaurants within 2 km of the coordinates (52.5200, 13.4050) in Berlin:

1. **Der Alte Fritz**
   - **Cuisine:** German
   - **Address:** Panoramastraße 1, 10178 Berlin
   - **Distance:** 0.33 km
   - **Phone:** +49 30 96200600
   - **Website:** [der-alte-fritz.eatbu.com](https://der-alte-fritz.eatbu.com/)
   - **Opening Hours:** PH, Mo-Su 12:00-22:00
   - **Outdoor Seating:** Yes

2. **Aapka - Restaurant Alexanderplatz**
   - **Cuisine:** Indian
   - **Address:** Karl-Liebknecht-Straße 23, 10178 Berlin
   - **Distance:** 0.57 km
   - **Phone:** +49 30 85077788
   - **Website:** [aapka.de](https://www.aapka.de/restaurants/berlin-alexanderplatz/)
   - **Opening Hours:** 11:30+
   -

### 🔍 Test 6: suggest_places - Get Place Suggestions

In [12]:
# Test suggest_places tool
print("🔍 Testing: suggest_places")
print("=" * 60)

query = "Suggest places matching 'Brandenburg Gate'"

print(f"📝 Query: {query}\n")
response = assistant.chat(query, verbose=True)
print(f"\n✅ Response:\n{response}")

🔍 Testing: suggest_places
📝 Query: Suggest places matching 'Brandenburg Gate'

🔧 Calling 1 tool(s)...
   → suggest_places(['query'])

✅ Response:
Here are some places matching 'Brandenburg Gate':

1. **Brandenburger Tor**
   - **Category:** Attraction
   - **Location:** Berlin, Germany
   - **Coordinates:** Latitude 52.51627, Longitude 13.37770
   - **Description:** A historic monument located at Pariser Platz in the Mitte district of Berlin.

2. **Brandenburger Tor** (Historic Monument)
   - **Category:** Historic Monument
   - **Location:** Berlin, Germany
   - **Coordinates:** Latitude 52.51627, Longitude 13.37770
   - **Description:** This refers to the same structure and is noted for its significance as a symbol of Berlin.

3. **Бранденбургские ворота (Brandenburg Gates)**
   - **Category:** Attraction
   - **Location:** Kaliningrad, Russia
   - **Coordinates:** Latitude 54.69728, Longitude 20.49468
   - **Description:** This is a historic monument in Kaliningrad, Russia, represen

---

## 🔗 Complex Multi-Tool Test

Now let's test a complex query that uses multiple tools in sequence!

### 🎯 Complex Query: Combining 4 Tools

In [13]:
# Complex multi-tool query
print("🔍 Testing: COMPLEX MULTI-TOOL QUERY")
print("=" * 80)

query = "Find cafes near the Eiffel Tower and give me walking directions from the Louvre"

print(f"📝 Query: {query}")
print(f"🎯 Expected Tools: forward_geocode (2x) → search_nearby → route")
print(f"📊 Expected Tool Count: 4\n")

print("🔧 Tool Calls:\n")
response = assistant.chat(query, verbose=True)

print("\n" + "=" * 80)
print(f"✅ Final Response:\n{response}")
print("=" * 80)

🔍 Testing: COMPLEX MULTI-TOOL QUERY
📝 Query: Find cafes near the Eiffel Tower and give me walking directions from the Louvre
🎯 Expected Tools: forward_geocode (2x) → search_nearby → route
📊 Expected Tool Count: 4

🔧 Tool Calls:

🔧 Calling 1 tool(s)...
   → search_nearby(['latitude', 'longitude', 'category', 'radius_km'])
🔧 Calling 1 tool(s)...
   → route(['origin', 'destination', 'mode'])

✅ Final Response:
Here are some cafes near the Eiffel Tower along with walking directions from the Louvre:

### Cafes Near the Eiffel Tower:
1. **Café de l'Alma**
   - Distance: 0.57 km
   - Phone: +33 1 45 51 56 74
   - [Facebook](https://www.facebook.com/Caf%C3%A9-de-lalma-350090191753899/)

2. **Le Campanella**
   - Distance: 0.65 km
   - Phone: +33 1 45 55 73 15
   - Opening Hours: Mo-Sa 08:00-23:00; Su 09:00-23:00

3. **L’Ancien Trocadéro**
   - Distance: 0.69 km

4. **Le Coq**
   - Distance: 0.70 km

5. **Café Kléber**
   - Distance: 0.74 km
   - Phone: +33 1 47 27 86 65
   - [Website](http://w

### 🌟 Another Complex Example: 5 Tools

This query demonstrates the maximum complexity with 5 tool calls!

In [14]:
# Maximum complexity multi-tool query
print("🔍 Testing: MAXIMUM COMPLEXITY QUERY")
print("=" * 80)

query = "What's at coordinates 48.8584, 2.2945, find restaurants within 1km, and tell me how to get there from Arc de Triomphe"

print(f"📝 Query: {query}")
print(f"🎯 Expected Tools: reverse_geocode → search_nearby → suggest_places → forward_geocode → route")
print(f"📊 Expected Tool Count: 5\n")

print("🔧 Tool Calls:\n")
response = assistant.chat(query, verbose=True)

print("\n" + "=" * 80)
print(f"✅ Final Response:\n{response}")
print("=" * 80)
print("\n💡 This demonstrates the agent's ability to orchestrate multiple tools intelligently!")

🔍 Testing: MAXIMUM COMPLEXITY QUERY
📝 Query: What's at coordinates 48.8584, 2.2945, find restaurants within 1km, and tell me how to get there from Arc de Triomphe
🎯 Expected Tools: reverse_geocode → search_nearby → suggest_places → forward_geocode → route
📊 Expected Tool Count: 5

🔧 Tool Calls:

🔧 Calling 3 tool(s)...
   → reverse_geocode(['latitude', 'longitude'])
   → search_nearby(['latitude', 'longitude', 'category', 'radius_km'])
   → reverse_geocode(['latitude', 'longitude'])

✅ Final Response:
### Location Information
The coordinates **48.8584, 2.2945** correspond to **Avenue Gustave Eiffel** in the **7th Arrondissement** of Paris, near the Eiffel Tower.

### Nearby Restaurants (within 1 km)
Here are some restaurants within 1 km of the Eiffel Tower:

1. **Linette** (French)
   - Distance: 0.48 km
   - Location: 48.8600, 2.3006

2. **OKA Sushi** (Japanese)
   - Distance: 0.73 km
   - Location: 48.8567, 2.3042

3. **La Maison du Sushi** (Sushi)
   - Distance: 0.76 km
   - Location

## 🎨 Step 7: Interactive Gradio Interface

In [15]:
import gradio as gr
import io
import sys
from contextlib import redirect_stdout

# Initialize assistant
assistant = MapAssistant()

# Example queries
example_queries = [
    ["Find me a cafe near the Eiffel Tower"],
    ["What's the address of coordinates 48.8584, 2.2945?"],
    ["Get walking directions from Notre-Dame to the Louvre in Paris"],
    ["Search for museums in Berlin"],
    ["Find restaurants within 2km of coordinates 52.5200, 13.4050"],
]

def process_query(query, verbose):
    """Process the user's query and return formatted response"""
    if not query.strip():
        return "⚠️ Please enter a query!"

    try:
        # Create status message with better formatting
        output_parts = []
        output_parts.append(f"### 🗺️ Your Query\n{query}\n")

        if verbose:
            output_parts.append("\n### 🔧 Processing Steps\n")
            # Capture verbose output
            verbose_buffer = io.StringIO()

            # Temporarily redirect stdout to capture print statements
            old_stdout = sys.stdout
            sys.stdout = verbose_buffer

            try:
                response = assistant.chat(query, verbose=True)
            finally:
                sys.stdout = old_stdout

            # Get captured verbose output
            verbose_output = verbose_buffer.getvalue()
            if verbose_output:
                output_parts.append(f"```\n{verbose_output}```\n")
        else:
            response = assistant.chat(query, verbose=False)

        # Format the final response
        output_parts.append(f"\n### 🤖 Assistant Response\n{response}")

        return "\n".join(output_parts)

    except Exception as e:
        return f"### ❌ Error Occurred\n\n```python\n{str(e)}\n```"

# Custom CSS for beautiful design with better contrast
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
#output_box {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    color: white !important;
    padding: 25px !important;
    border-radius: 12px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
}
#output_box * {
    color: white !important;
}
#output_box h3 {
    color: #fff !important;
    font-weight: bold !important;
    margin-top: 15px !important;
    margin-bottom: 10px !important;
    border-bottom: 2px solid rgba(255, 255, 255, 0.3) !important;
    padding-bottom: 5px !important;
}
#output_box code {
    background: rgba(0, 0, 0, 0.3) !important;
    padding: 2px 6px !important;
    border-radius: 4px !important;
    color: #fff !important;
}
#output_box pre {
    background: rgba(0, 0, 0, 0.3) !important;
    padding: 15px !important;
    border-radius: 8px !important;
    overflow-x: auto !important;
    border: 1px solid rgba(255, 255, 255, 0.2) !important;
}
#output_box pre code {
    background: transparent !important;
}
.input-text textarea {
    border: 2px solid #667eea !important;
    border-radius: 8px !important;
}
.input-text textarea:focus {
    border-color: #764ba2 !important;
    box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
}
"""

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="🗺️ MCP Map Servers") as demo:

    gr.Markdown("""
    # 🗺️ MCP Map Servers - AI Navigation Assistant

    ### Your intelligent map companion powered by OpenAI and OpenStreetMap

    Ask me anything about:
    - 📍 **Geocoding** - Convert addresses to coordinates and vice versa
    - 🚗 **Routing** - Get directions with multiple travel modes
    - 🏛️ **POI Search** - Find cafes, restaurants, museums, and more
    - 🌍 **Location Discovery** - Explore places around the world
    """)

    with gr.Row():
        with gr.Column(scale=2):
            query_input = gr.Textbox(
                label="🔍 Enter Your Query",
                placeholder="e.g., Find me a cafe near the Eiffel Tower",
                lines=3,
                elem_classes="input-text"
            )

            verbose_checkbox = gr.Checkbox(
                label="🔧 Verbose Mode (Show tool calls and processing steps)",
                value=False
            )

            with gr.Row():
                submit_btn = gr.Button("🗺️ Ask Assistant", variant="primary", scale=2)
                clear_btn = gr.ClearButton([query_input], value="🗑️ Clear", scale=1)

    gr.Markdown("### 💡 Try These Examples:")

    with gr.Row():
        for example in example_queries[:3]:
            gr.Button(example[0][:30] + "...", size="sm").click(
                fn=lambda x=example[0]: x,
                outputs=query_input
            )

    with gr.Row():
        for example in example_queries[3:]:
            gr.Button(example[0][:30] + "...", size="sm").click(
                fn=lambda x=example[0]: x,
                outputs=query_input
            )

    gr.Markdown("---")

    output_display = gr.Markdown(
        label="📋 Response",
        elem_id="output_box",
        value="*Your responses will appear here...*"
    )

    # Event handlers with loading state
    submit_btn.click(
        fn=process_query,
        inputs=[query_input, verbose_checkbox],
        outputs=output_display,
        api_name="query"
    ).then(
        fn=lambda: gr.Button("🗺️ Ask Assistant", interactive=True),
        outputs=submit_btn
    )

    # Also handle Enter key submission
    query_input.submit(
        fn=process_query,
        inputs=[query_input, verbose_checkbox],
        outputs=output_display
    )

    gr.Markdown("""
    ---
    ### 📚 Tips:
    - Use **natural language** - the AI understands conversational queries
    - Enable **Verbose Mode** to see which tools are being called
    - Try **combining requests** - e.g., "Find a cafe near X and give me directions from Y"
    - Use **coordinates** for precise locations - format: latitude, longitude

    ### 🛠️ Available Tools:
    - `forward_geocode` - Address → Coordinates. **Example:** What are the coordinates of the Eiffel Tower?
    - `reverse_geocode` - Coordinates → Address. **Example:** What's located at 34.0522, -118.2437
    - `route` - Get directions (driving/walking/cycling).  **Example:** Get walking directions from Notre-Dame to the Louvre in Paris
    - `search_poi` - Find places by category in a city.  **Example:** Find museums in Rome
    - `search_nearby` - Find places near coordinates  **Example:** Find cafes near the Eiffel Tower
    - `suggest_places` - Get location suggestions. **Example:** Suggest places matching 'Arc de Triomphe'

    ### 🔗 Combining Requests — Examples

    - *Suggest places matching “Arc de Triomphe,” find cafés within 500 m, and give me walking directions from the Champs-Élysées.*
    - *What's located at `34.0522, -118.2437`, find movie theaters within 3 km, and tell me how to bike there from Hollywood Boulevard.*
    - *Find hotels near Tokyo Tower, get the address of the first one, and give me driving directions from Shibuya Crossing.*
    - *Help me plan a trip: start at Trafalgar Square, find museums within 1 km, and give me walking directions to the British Museum.*

    """)

# Launch the interface
print("✅ Launching Gradio interface...")
demo.launch(debug=True, share=False)

✅ Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
